## Example 01: CDFT calculation of the electronic coupling of $\mathrm{He}_2^+$

### Initialize DFT driver (Qbox)

After the installation of PyCDFT and a DFT driver (Qbox in this example), perform the ground state DFT calculation with Qbox.

```bash
   export qb="/path/to/qbox"
   $mpirun -np <ntasks> $qb < gs.in > gs.out
```
where /path/to/qbox is the path to the Qbox executable and ntasks denotes the number of MPI processes.

Then in the same directory, execute Qbox in [client-server mode](qboxcode.org/daoc/html/usage/client-server.html)

```bash
   $mpirun -np <ntasks> $qb -server qb_cdft.in qb_cdft.out
```
and leave the terminal open throughout the entire calculation, Qbox will response to commands given by PyCDFT.

Make sure this Jupyter notebook sits in the same directory as the ground state calculation.

### Read atomic structure of $\mathrm{He}_2^+$

In [ ]:
from pycdft import *
from ase.io import read

# read atomic structure using ASE
cell = read("./He2_3Ang.cif")

# construct the Sample instance
sample = Sample(ase_cell=cell, n1=112, n2=112, n3=112, vspin=1)

### Set up CDFT calculations

First, construct a **DFTDriver** that provide necessary commands to for PyCDFT to communicate with Qbox

In [ ]:
qboxdriver = QboxDriver(
    sample=sample,
    init_cmd="load gs.xml \n" 
        "set xc PBE \n" 
        "set wf_dyn PSDA \n" 
        "set_atoms_dyn CG \n" 
        "set scf_tol 1.0E-8 \n",
    scf_cmd="run 0 50 5",
)

Then, set up **CDFTSolver** instances, which orchestrate the entire CDFT calculations.
In order to compute the electronic coupling, we need two solvers for the two diabatic state where the extra +1 charge is localized on each He atom.

In [ ]:
solver1 = CDFTSolver(
    job="scf",  # indicate that the calculation is a SCF calculation
    optimizer="brenth", # specifiy the optimizer used for the Lagrangian multiplier
    sample=sample, dft_driver=qboxdriver
)
solver2 = solver1.copy()

Finally, we add constraints to the solvers by constructing **Constraint** instances.
In this example we will use **ChargeTransferConstraint** to enforce the electron number difference between the two He atoms to be 1.

In [ ]:
V = (2,-2)

ChargeTransferConstraint(
    sample=solver1.sample,
    donor=Fragment(solver1.sample, solver1.sample.atoms[0:1]), # donor fragment
    acceptor=Fragment(solver1.sample, solver1.sample.atoms[1:2]), # acceptor fragment
    V_brak =V, # search region for the brenth optimizer
    N0=1, # desired charge to be localized
    N_tol=1E-6 # convergence threshold for |N - N0|
)
ChargeTransferConstraint(
    sample=solver2.sample, 
    donor=Fragment(solver2.sample, solver2.sample.atoms[0:1]),
    acceptor=Fragment(solver2.sample, solver2.sample.atoms[1:2]),
    V_brak=V,
    N0=-1, 
    N_tol=1E-6
)

In this example we used V = (2,-2) as the search region for the **brenth** optimizer, which we found to be sufficient for most cases.

Note that if one has a good guess for the Lagrange multipliers in constraint potentials (for instance from previous calculations using smaller kinetic energy cutoff, etc.), it is recommended to use optimizers such as **secant**, which can take a initial guess for the Lagrange multiplier. In this case, the **V_brak** parameter should be replaced by the **V_init** parameter when initializing the constraints.
For the He-He+ dimer separated by 3 Angstrom, a good starting guess is V_init = -0.7 for solver1 and V_init = 0.7 for solver2.

### Execute CDFT calculations

Call the **solve** method of **CDFTSolver** instances to execute CDFT calculations.

In [ ]:
solver1.solve()

In [ ]:
solver2.solve()

Finally, we call upon the routines for calculating electronic coupling. 
An example output is given in ./reference

### Compute electronic coupling

After CDFT calculations are converged, the electronic coupling Hab can be computed using the **compute_elcoupling** function

In [ ]:
compute_elcoupling(solver1, solver2)